# Lesson 2 Database Access

In this lesson, we are going to learn simple database access in Python. We are going to use SQLite since it doesn't require much setup. 

SQLite is a lightweight database that is stores the database in a file without having to run a dedicated server. The advantage is that SQLite is very lightweight and doesn't require any setup as long as you have the file. The disadvantage is that it is not designed for scalable concurrent access, so it is not suitable for server-side production system that is used by many users.

The example is adapted from the official documentation at https://docs.python.org/3.7/library/sqlite3.html

To use the module, you must first create a Connection object that represents the database. Here the data will be stored in the example.db file. 

If you don't wish to create any file (e.g. for testing purpose) you can ask SQLite to keep the database in memory by using :memory: instead. If you use memory, the data will be wiped out when you close the connection, so make sure that you don't close the connection until you are done with your testing. 

In [1]:
import sqlite3
conn = sqlite3.connect('example.db')
#conn = sqlite3.connect(':memory:')

We need to create a cursor object from the connection. We use cursor object to execute SQL instructions. 

In [2]:
c = conn.cursor()
print (type(conn), type(c))

<class 'sqlite3.Connection'> <class 'sqlite3.Cursor'>


Creating a table using cursor object. Note the use of triple single-quote to write a multi-line string.

In [3]:
# Create table
c.execute('''CREATE TABLE stocks
             (date text, trans text, symbol text, qty real, price real)''')


In [4]:
# Insert a row of data
c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
conn.commit()

You shouldn’t assemble your query using Python’s string operations because doing so is insecure; it makes your program vulnerable to an SQL injection attack (see https://xkcd.com/327/ for humorous example of what can go wrong).

We should use parameter substitution instead of assembling the SQL using string operation.

In [5]:
# Larger example that inserts many records at a time
purchases = [('2006-03-28', 'BUY', 'IBM', 1000, 45.00),
             ('2006-04-05', 'BUY', 'MSFT', 1000, 72.00),
             ('2006-04-06', 'SELL', 'IBM', 500, 53.00),
            ]
c.executemany('INSERT INTO stocks VALUES (?,?,?,?,?)', purchases)

conn.commit()

We need to close the connection once we are done with it. This is especially important when building long-running application (like a web application) to avoid 'connection leak' problem. Connection leak is a term used to describe a problem of an application running out of database connections because modules do not close the connection after they are done with it.

In [6]:
# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

In [7]:
import sqlite3
conn = sqlite3.connect('example.db')
c = conn.cursor()

In [47]:
# Never do this -- insecure!
symbol = 'RHAT'
c.execute("SELECT * FROM stocks WHERE symbol = '%s'" % symbol)

# Do this instead
t = ('RHAT',)
c.execute('SELECT * FROM stocks WHERE symbol=?', t)
print(c.fetchone())

#list of columns returned
print(c.description) # this returned a troublesome return value

#we can do this to just read the column name
for d in c.description:
    print(d[0])

#or if we have learned about list comprehension we can shorten it this way.
print([x[0] for x in c.description])

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)
(('date', None, None, None, None, None, None), ('trans', None, None, None, None, None, None), ('symbol', None, None, None, None, None, None), ('qty', None, None, None, None, None, None), ('price', None, None, None, None, None, None))
date
trans
symbol
qty
price
['date', 'trans', 'symbol', 'qty', 'price']


In [35]:
for row in c.execute('SELECT * FROM stocks ORDER BY price'):
        print(row)


('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)
('2006-03-28', 'BUY', 'IBM', 1000.0, 45.0)
('2006-04-06', 'SELL', 'IBM', 500.0, 53.0)
('2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0)


In [34]:
rows = c.execute('SELECT * FROM stocks ORDER BY price').fetchall()
print(type(rows), len(rows))

for row in rows:
#    print(type(row))
    print(row[0])


<class 'list'> 4
2006-01-05
2006-03-28
2006-04-06
2006-04-05


Always close connection at the end.

In [49]:
conn.close()
